# Studying the Fuel Economy of Cars Over Time
**Ryan Eckenrod, Nathaniel Foote, Erica Phlek  
CMSC320 Final Tutorial  
5/19/17**


An ever growing body of scientific research supports the theory that greenhouse gases emitted by human activity are [dangerously warming Earth's atomosphere.](https://climate.nasa.gov/evidence/) Modern transportation, often achieved by burning fossil fuels, is a significant contributor of these greenhouse gases, namely carbon dioxide (CO<sub>2</sub>). [The most recent estimates](https://www.epa.gov/ghgemissions/sources-greenhouse-gas-emissions) from the Environmental Protection Agency (EPA) indicate that transportation accounts for 27% of all greenhouse gas emissions in the United States, second only to electricity production. Of transportation emissions, [61% come from privately owned passenger cars and trucks,](https://www.epa.gov/greenvehicles/fast-facts-transportation-greenhouse-gas-emissions) which amounts to 16.5% of all American emissions.  

Given the United States' [large emission footprint globally](https://www.epa.gov/ghgemissions/global-greenhouse-gas-emissions-data) and the significant contribution of greenhosue gases from private vehicles, the government is acting to reduce greenhouse gas emissions from cars and trucks sold in America. Under direction from President Obama in 2010, the EPA and the National Highway Traffic Safety Administration (NHTSA) [jointly developed new fuel economy standards](https://www.nhtsa.gov/laws-regulations/corporate-average-fuel-economy#9491) which car companies must meet in order to sell their vehicles. You can read more about the history, goals, and effectiveness of fuel economy standards in the United States and Europe [here](http://www.rff.org/files/document/file/RFF-PB-16-03.pdf), or about the Trump administration's efforts to review the latest standards [here](http://www.nbcnews.com/business/autos/trump-rolls-back-obama-era-fuel-economy-standards-n734256).  

With fuel economy standards increasing over time, it stands to reason to that cars and trucks should be getting more fuel efficient over time as well. Beyond that, simple economics suggests that consumers should prefer to purchase more fuel efficient vehicles. Why get a car with 15mpg that's expensive to drive when you can get one with 30mpg, right? Thus, in this tutorial we explore the fuel efficiency of cars from 1984 to 2017. We obtained a CSV file with fuel economy measures for over 37,000 vehicle models sold in the United States since 1984 from [Kapsarc](https://datasource.kapsarc.org/page/about/), an energy focused data science portal which last scraped the data on 4/30/17 from [fueleconomy.gov](https://www.fueleconomy.gov/feg/ws/index.shtml), an EPA hosted site. How has the average highway mpg of vehicles changed over time compared to city mpg? Did the 2010 fuel economy standards signficantly change the rate of improvement in vehicle fuel efficieny? We'll aim to answer these and other questions while we explore this [dataset](https://datasource.kapsarc.org/explore/dataset/us-vehicle-fuel-economy-data-1984-2017/table/).  

### I: Importing and Cleaning the Dataset

In [71]:
#Let's first import some useful python libraries
import pandas as pd
import seaborn as sns
import numpy as np

In [72]:
#Next, let's import the dataset and take a quick view at its contents
df = pd.read_csv("us-vehicle-fuel-economy-data-1984-2017.csv", sep=';')
pd.set_option('display.max_columns', None)
print(df.shape)
df.head()

(37129, 83)


,barrels08,barrelsA08,charge120,charge240,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,cylinders,displ,drive,engId,eng_dscr,feScore,fuelCost08,fuelCostA08,fuelType,fuelType1,ghgScore,ghgScoreA,highway08,highway08U,highwayA08,highwayA08U,VClass,highwayCD,highwayE,highwayUF,hlv,hpv,id,lv2,lv4,make,model,mpgData,phevBlended,pv2,pv4,range,rangeCity,rangeCityA,rangeHwy,rangeHwyA,trany,UCity,UCityA,UHighway,UHighwayA,year,youSaveSpend,guzzler,trans_dscr,tCharger,sCharger,atvType,fuelType2,rangeA,evMotor,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,15.695714,0.0,0.0,0.0,18,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,4.0,2.0,Front-Wheel Drive,59007,(FFS),-1,1600,0,Regular,Regular Gasoline,-1,-1,26,0.0,0,0.0,Compact Cars,0.0,0.0,0.0,0,0,10009,0,15,Volkswagen,Jetta III,N,False,0,88,0,0.0,0.0,0.0,0.0,Automatic 4-spd,23.0000,0.0,36.0000,0.0,1993,-1250,NaN,2MODE CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0
1,17.347895,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,467.736842,19,0.0,0,0.0,0.0,0.0,0.0,6.0,3.3,Front-Wheel Drive,4410,(FFS),-1,1750,0,Regular,Regular Gasoline,-1,-1,24,0.0,0,0.0,Midsize Cars,0.0,0.0,0.0,0,0,10019,16,16,Buick,Century,Y,False,98,97,0,0.0,0.0,0.0,0.0,Automatic 3-spd,21.1111,0.0,33.0000,0.0,1993,-2000,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0
2,20.600625,0.0,0.0,0.0,14,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,555.437500,16,0.0,0,0.0,0.0,0.0,0.0,8.0,4.0,Rear-Wheel Drive,12071,(GUZZLER) (FFS),-1,2500,0,Premium,Premium Gasoline,-1,-1,20,0.0,0,0.0,Midsize Cars,0.0,0.0,0.0,0,0,10015,0,13,BMW,740i,N,False,0,101,0,0.0,0.0,0.0,0.0,Automatic 5-spd,17.7778,0.0,28.0000,0.0,1993,-5750,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0
3,16.480500,0.0,0.0,0.0,18,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,444.350000,20,0.0,0,0.0,0.0,0.0,0.0,6.0,3.3,Front-Wheel Drive,2611,(FFS),-1,1650,0,Regular,Regular Gasoline,-1,-1,24,0.0,0,0.0,Midsize Cars,0.0,0.0,0.0,0,0,10042,0,16,Dodge,Dynasty,N,False,0,99,0,0.0,0.0,0.0,0.0,Automatic 4-spd,22.0000,0.0,33.3333,0.0,1993,-1500,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0
4,16.480500,0.0,0.0,0.0,17,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,444.350000,20,0.0,0,0.0,0.0,0.0,0.0,6.0,3.8,Front-Wheel Drive,4400,(FFS),-1,1650,0,Regular,Regular Gasoline,-1,-1,26,0.0,0,0.0,Midsize Cars,0.0,0.0,0.0,0,0,10023,16,16,Buick,Regal,Y,False,95,100,0,0.0,0.0,0.0,0.0,Automatic 4-spd,21.0000,0.0,36.0000,0.0,1993,-1500,NaN,CLKUP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,2013-01-01,2013-01-01,NaN,0,0,0


If you didn't check out the dataset link in the introduction, the full description of the dataset and every column's meaning can be found [here.](https://datasource.kapsarc.org/explore/dataset/us-vehicle-fuel-economy-data-1984-2017/information/) There are 37,129 vehicles with 83 attributes in the dataset. Looking over the .head() output, the columns of the dataset appear to be arranged in no particular order. Scrolling through them suggests that each row corresponds to a unique vehicle model via the primary key "id", which roughly corresponds to a unique combination of the attributes "make", "model", "year," "trany" (indicating transmission type), and "fuelType1/2" (e.g. regular gasoline, electric, or diesel). The first thing we'll do then is reorganize the columns so these attributes appear at the beginning of the dataset and so the rest follow alphabetically.

In [73]:
#resort the columns by identifying information at the front, then alphabetically
head = ['id', 'make', 'model', 'year', 'trany', 'fuelType1', 'fuelType2']
cols = [x for x in sorted((df)) if x not in head]
cols = head + cols
df = df[cols].sort_values(by='id')

#view the newly organized dataset
df.head()

,id,make,model,year,trany,fuelType1,fuelType2,UCity,UCityA,UHighway,UHighwayA,VClass,atvType,barrels08,barrelsA08,c240Dscr,c240bDscr,charge120,charge240,charge240b,city08,city08U,cityA08,cityA08U,cityCD,cityE,cityUF,co2,co2A,co2TailpipeAGpm,co2TailpipeGpm,comb08,comb08U,combA08,combA08U,combE,combinedCD,combinedUF,createdOn,cylinders,displ,drive,engId,eng_dscr,evMotor,feScore,fuelCost08,fuelCostA08,fuelType,ghgScore,ghgScoreA,guzzler,highway08,highway08U,highwayA08,highwayA08U,highwayCD,highwayE,highwayUF,hlv,hpv,lv2,lv4,mfrCode,modifiedOn,mpgData,phevBlended,phevCity,phevComb,phevHwy,pv2,pv4,range,rangeA,rangeCity,rangeCityA,rangeHwy,rangeHwyA,sCharger,startStop,tCharger,trans_dscr,youSaveSpend
33941,1,Alfa Romeo,Spider Veloce 2000,1985,Manual 5-spd,Regular Gasoline,NaN,23.3333,0.0,35.000,0.0,Two Seaters,NaN,15.695714,0.0,NaN,NaN,0.0,0.0,0.0,19,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,423.190476,21,0.0,0,0.0,0.0,0.0,0.0,2013-01-01,4.0,2.0,Rear-Wheel Drive,9011,(FFS),NaN,-1,1600,0,Regular,-1,-1,NaN,25,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN,2013-01-01,Y,False,0,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,-1250
2979,2,Bertone,X1/9,1985,Manual 5-spd,Regular Gasoline,NaN,25.0000,0.0,36.000,0.0,Two Seaters,NaN,14.982273,0.0,NaN,NaN,0.0,0.0,0.0,20,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,403.954545,22,0.0,0,0.0,0.0,0.0,0.0,2013-01-01,4.0,1.5,Rear-Wheel Drive,12710,NaN,NaN,-1,1500,0,Regular,-1,-1,NaN,26,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN,2013-01-01,Y,False,0,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,-750
27386,3,Chevrolet,Corvette,1985,Automatic 4-spd,Regular Gasoline,NaN,18.0000,0.0,29.000,0.0,Two Seaters,NaN,19.388824,0.0,NaN,NaN,0.0,0.0,0.0,15,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,522.764706,17,0.0,0,0.0,0.0,0.0,0.0,2013-01-01,8.0,5.7,Rear-Wheel Drive,4185,(350 V8) (FFS),NaN,-1,1950,0,Regular,-1,-1,NaN,21,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN,2013-01-01,Y,False,0,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,-3000
6311,4,Chevrolet,Corvette,1985,Manual 4-spd,Regular Gasoline,NaN,18.0000,0.0,28.000,0.0,Two Seaters,NaN,19.388824,0.0,NaN,NaN,0.0,0.0,0.0,15,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,522.764706,17,0.0,0,0.0,0.0,0.0,0.0,2013-01-01,8.0,5.7,Rear-Wheel Drive,4185,(350 V8) (FFS),NaN,-1,1950,0,Regular,-1,-1,NaN,20,0.0,0,0.0,0.0,0.0,0.0,0,0,0,0,NaN,2013-01-01,N,False,0,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,-3000
14021,5,Nissan,300ZX,1985,Automatic 4-spd,Regular Gasoline,NaN,18.0000,0.0,24.359,0.0,Two Seaters,NaN,20.600625,0.0,NaN,NaN,0.0,0.0,0.0,15,0.0,0,0.0,0.0,0.0,0.0,-1,-1,0.0,555.437500,16,0.0,0,0.0,0.0,0.0,0.0,2013-01-01,6.0,3.0,Rear-Wheel Drive,38043,"(GUZZLER) (FFS,TRBO)",NaN,-1,2100,0,Regular,-1,-1,T,18,0.0,0,0.0,0.0,0.0,0.0,23,50,0,0,NaN,2013-01-01,N,False,0,0,0,0,0,0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,T,2MODE,-3750


That's nicer! Still, we have a very large number of attributes, and it also appears that many of the columns of the dataset are mostly empty with values of 0, -1, or NaN. We don't feel trying to impute missing values for such attributes would be appropriate given our lack of understanding about the distribution from which the measurements are drawn. Some attributes are also redundant in the information they record; "barrels08" for instance logs the annual petroleum consumption in barrels of a car, but this is a direct function of its "comb08" (combined city/hwy mpg) attribute. Another example is "UCity" and "UHighway", which are [outdated mpg estimates](https://www.fueleconomy.gov/feg/ratings.shtml) compared to "city08" and "highway08." Other attributes such as "createdOn" record information about when the vehicle was added to the dataset which isn't a useful attribute for our analysis. 

For these reasons, we'd like to reduce the number of attributes down to those we are interested in, those which aren't redundant, and those which are not mostly empty-values for the entire dataset. We thus remove the attributes listed below, as well as any attributes we're not interested in for which less than 1% of cars have a non-empty value of the attrbute.

In [74]:
#Redundant or non-useful attributes we'll remove:
#We remove fuelCost attributes on the basis that fuel prices change over time,
#so these attributes only encapsulate the fuelCost for a given time.
drop = ['UCity', 'UCityA', 'atvType', 'barrels08', 'barrelsA08', 'fuelType', \
       'UHighway', 'UHighwayA', 'createdOn', 'modifiedOn', 'cityA08U', 'city08U', \
       'co2', 'co2A', 'comb08U', 'combA08U', 'engId', 'eng_dscr', 'fuelCost08', \
       'fuelCostA08', 'guzzler', 'highway08U', 'highwayA08U', 'mfrCode', \
       'mpgData', 'trans_dscr', 'youSaveSpend', 'startStop']

#collect columns which are are mostly empty
empty_attrs = []
empty_vals = [np.NaN, 0, -1]
for col_name in cols:
    col = df[col_name]
    non_empty = [x for x in col if x not in empty_vals]
    #print(col_name, len(non_empty) / len(col))
    if len(non_empty) / len(col) < 0.01:
        empty_attrs.append(col_name)
        
#remove unwanted attributes
drop = list(set().union(empty_attrs, drop))
df = df.drop(drop, axis = 1)
df.head()

,id,make,model,year,trany,fuelType1,fuelType2,VClass,city08,cityA08,co2TailpipeAGpm,co2TailpipeGpm,comb08,combA08,cylinders,displ,drive,evMotor,feScore,ghgScore,ghgScoreA,highway08,highwayA08,hlv,hpv,lv2,lv4,pv2,pv4,rangeA,sCharger,tCharger
33941,1,Alfa Romeo,Spider Veloce 2000,1985,Manual 5-spd,Regular Gasoline,NaN,Two Seaters,19,0,0.0,423.190476,21,0,4.0,2.0,Rear-Wheel Drive,NaN,-1,-1,-1,25,0,0,0,0,0,0,0,NaN,NaN,NaN
2979,2,Bertone,X1/9,1985,Manual 5-spd,Regular Gasoline,NaN,Two Seaters,20,0,0.0,403.954545,22,0,4.0,1.5,Rear-Wheel Drive,NaN,-1,-1,-1,26,0,0,0,0,0,0,0,NaN,NaN,NaN
27386,3,Chevrolet,Corvette,1985,Automatic 4-spd,Regular Gasoline,NaN,Two Seaters,15,0,0.0,522.764706,17,0,8.0,5.7,Rear-Wheel Drive,NaN,-1,-1,-1,21,0,0,0,0,0,0,0,NaN,NaN,NaN
6311,4,Chevrolet,Corvette,1985,Manual 4-spd,Regular Gasoline,NaN,Two Seaters,15,0,0.0,522.764706,17,0,8.0,5.7,Rear-Wheel Drive,NaN,-1,-1,-1,20,0,0,0,0,0,0,0,NaN,NaN,NaN
14021,5,Nissan,300ZX,1985,Automatic 4-spd,Regular Gasoline,NaN,Two Seaters,15,0,0.0,555.437500,16,0,6.0,3.0,Rear-Wheel Drive,NaN,-1,-1,-1,18,0,23,50,0,0,0,0,NaN,NaN,T


In [ ]:
#